***Expanding Nodes and maintainig Frontier***

Valle Varo

**The problem "Wolf, goat, and cabbage"**

https://en.wikipedia.org/wiki/River_crossing_puzzle

A farmer must transport a wolf, a goat and cabbage from one side of a river to another using a boat which can only hold one item in addition to the farmer, subject to the constraints that the wolf cannot be left alone with the goat,  and the goat cannot be left alone with the cabbage

## State definition and comprobations
The state will be defined as a 4-elements (boolean) vector, representing each one of them the position of the elements:

1st position: location of the farmer (LEFT, RIGHT)

2nd position: location of the wolf (LEFT, RIGHT)

3rd position: location of the goat (LEFT, RIGHT)

4th position: location of the cabbage (LEFT, RIGHT)

In [1]:
state = {"Farmer":"L","Wolf":"L","Goat":"L","Cabbage":"L"}
state

{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}


### Final state comprobation
In this problem the checking if a state is the final one (the SOLUTION), is just the comprobation all the elements of the vector are equal to R



In [2]:
#change one element to R to move it to the other side
state = {"Farmer":"L","Wolf":"L","Goat":"L","Cabbage":"L"}
state["Farmer"]="R"
print (state)
comprobation = all(value == "R" for value in state.values())
print (comprobation)

state = {"Farmer":"R","Wolf":"R","Goat":"R","Cabbage":"R"}
print (state)
comprobation =  all(value == "R" for value in state.values())
print (comprobation)

{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
False
{'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'R'}
True


##Actions, conditions and effects

Now we have an initial state, and a way for checking we are or not in the final one, we can define the way the agent will move in the environment.

4 different actions can be done
- 1: "farmer"  -> changes the side of the farmer (alone)
- 2: "wolf"    -> changes the side of the wolf (and also the farmer)
- 3: "goat"    -> changes the side of the goat (and also the farmer)
- 4: "cabbage" -> changes the side of the cabbage (and also the farmer)

Considering: moving the farmer can be done if no restrictions are violated. Moving another element can be done if no restrictions are violated and the element is in the same side of the river than the farmer

### Restrictions

For moving the farmer, conditions to be satisfied by the state are wolf and goat are not left alone, as well as goat and cabbage cannot be left togheter. With the goat and the wolf the options are:

| Farmer | Wolf | Goat | OK |
| --- | --- | --- | --- |
| 0|0 |0 |OK |
| 0|0 |1 |OK |
| 0|1 |0 |OK |
| 0|1 |1 |NO OK |
| 1|0 |0 |NO OK |
| 1|0 |1 |OK |
| 1|1|0 |OK |
| 1|1 |1 |OK |

Just the cases where they are together and the farmer is in the contrary state. If they are together but the farmer is there, no probs!


In [3]:
#We define the state and make the comprobation by three different commands
#original state
state = {"Farmer":"L","Wolf":"L","Goat":"L","Cabbage":"L"}
#move farmer
state["Farmer"]="R"

#first restriction: wolf and goat / goat and cabbage cannot be together without the farmer
comprobation_wolf_goat = not ((state["Wolf"]==state["Goat"]) & (state["Wolf"]!=state["Farmer"]))
comprobation_goat_cabbage = not ((state["Cabbage"]==state["Goat"]) & (state["Goat"]!=state["Farmer"]))
#these two conditions must be met together:
check_farmer_movement = comprobation_wolf_goat and comprobation_goat_cabbage
print (state)
print (check_farmer_movement)

# a function will do
def check_possible_state(state):
  comprobation_wolf_goat = not ((state["Wolf"]==state["Goat"]) & (state["Wolf"]!=state["Farmer"]))
  comprobation_goat_cabbage = not ((state["Cabbage"]==state["Goat"]) & (state["Goat"]!=state["Farmer"]))
  return comprobation_wolf_goat and comprobation_goat_cabbage

state["Farmer"]="L"
print (state)
print (check_possible_state(state))

#Check for different states
state["Farmer"]="R"
state["Wolf"]="R"
print (state)
print (check_possible_state(state))
#move them back
state["Wolf"]="L"
state["Farmer"]="L"

state["Farmer"]="R"
state["Goat"]="R"
print (state)
print (check_possible_state(state))

{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
False
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
True
{'Farmer': 'R', 'Wolf': 'R', 'Goat': 'L', 'Cabbage': 'L'}
False
{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
True


### ACTIONS: Move one element
Movement changes state but there are 2 main restrictions:
1. only farmer can move alone
2. if any of the other elements are moved, they move with farmer(need to be in the same side) and the new state is checked

If the movement is possible, new state is returned. If not,the same state as previous. This will allow to move new states to the frontier.

In [4]:
def change_element_status (element, state):
  if state[element]=="R":
        return "L"
  else:
        return "R"

def move (element, state):
  #check if more than 2 elements are moving:
  new_state=state.copy()### beware lists and dicts, if we dont use COPY, they just share the memory, altering one will alter the other one
  if element=="Farmer":
    #change the status to the new one in the new state
    new_state["Farmer"]=change_element_status(element, state)
    #check if this state is possible:
    if check_possible_state(new_state):
      return new_state
    else:
        print ("Can't move only farmer")
        return state #same as previous
  else: #if any other element is to be moved, first, check if it is in teh same side as the farmer
    if state[element]!=state["Farmer"]:
      print ("can't move "+element+" without farmer")
      return state #same as previous
    else:
      new_state["Farmer"]=change_element_status("Farmer", state)
      new_state[element]=change_element_status(element, state)
      #now, check if this new state is possible:
      if check_possible_state(new_state):
        return new_state
      else:
        print ("Can't move ", element)
        return state #same as previous

#Possible actions are defined with a list of the elements that can be moved
action_elements = ["Farmer", "Wolf", "Goat", "Cabbage"]



let's move some things


In [5]:
original_state = {"Farmer":"L","Wolf":"L","Goat":"L","Cabbage":"L"}

new_state = move("Farmer", original_state)
print (new_state)

Can't move only farmer
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}


In [6]:
new_state = move("Goat", new_state)
print (new_state)

{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}


In [7]:
new_state = move("Cabbage", original_state)
print (new_state)

Can't move  Cabbage
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}


In [8]:
new_state = move("Wolf", new_state)
print (new_state)

Can't move  Wolf
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}


seems to work.
Let's define a function to eval if the final solution has arrived, parting from the comprobation before:


In [9]:
def is_final_state (state):
  return all(value == "R" for value in state.values())



### Frontier
As we have seen, the frontier is encoded as a FIFO queue, that can be represented in Python as a simple list from where we can add or remove elements.

We define the frontier and the PUSH and PULL methods


In [10]:
frontier = []
def push_to_frontier (state, frontier):
  frontier.append(state)
  return frontier

def pull_from_frontier (frontier): #gets the first element, removes it from the frontier afterwards
  state = frontier[0]
  frontier = frontier[1:]
  return state, frontier

state = {'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'L'}
frontier =  push_to_frontier(state, frontier)
state = {'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'R'}
frontier =  push_to_frontier(state, frontier)
print (frontier)
state, frontier = pull_from_frontier(frontier)
print (state)
print (frontier)

[{'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'L'}, {'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'R'}]
{'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'L'}
[{'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'R'}]


### Expand nodes
Expanding nodes in this case is trying to move all agents from a particular state. Some movements are not possible, but this is already solved in the movement action script.





In [11]:
def expand (state, frontier):
  # check for every possible action which ones are possible, and add these
  # states to the frontier
  for element in action_elements:
    new_state = move (element, state)
    if new_state!=state:
      #if they are the same, the movement is forbidden.
      #If they are different, the new state is moved to the frontier
      frontier =  push_to_frontier(new_state, frontier)
  return frontier


#### LET'S TRy:
#original state and empty frontier
state = {'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
frontier = []
frontier  = expand(state, frontier)
print (frontier)

Can't move only farmer
Can't move  Wolf
Can't move  Cabbage
[{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}]


In [12]:
#now, we get a state from the frontier and expand it again:
new_state, frontier = pull_from_frontier(frontier)
frontier = expand (new_state, frontier)
print (frontier)

can't move Wolf without farmer
can't move Cabbage without farmer
[{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}, {'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}]


We can see that we have some repeated states. If we expand twice from the original state, we have the initial state again in the solution. We will solve it eventually with a graph formulation. But for now we will trust that BFS will find the solution even if it is not in the most efficient way (it is a complete algorithm).

## BFS
Let's assemble our very first BFS algorithm

```
1. Make a node with the initial problem state
2. Insert node into the frontier data structure
3. WHILE final state not found AND frontier is not empty DO
  3.1 Remove first node from the frontier
  3.2 IF node contains final state THEN final state found
  3.3 IF node doesn’t contain final state THEN
     3.3.1 EXPAND node’s state
     3.3.2 Insert successor nodes into frontier
4. IF final state found THEN
  4.1  RETURN sequence of actions found
5. ELSE  “solution not found”
```




In [14]:
# 1. problem definition
initial_state={'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
frontier = []

sequence_of_actions = [] #we create this list to store all the actions that the algorithm follows
state = {} # define state outside while context
# 2. add node to frontier
frontier =  push_to_frontier(initial_state, frontier)

# 3. start exploring and expanding the frontier
i=1
while len (frontier)>0: #if we have elements in the frontier...

  # 3.1. get first element of frontier and delete it
  state, frontier = pull_from_frontier (frontier)
  # **** add it to our sequence, to know which elements we have visited and check how is it going
  sequence_of_actions.append(state)
  print ("---------------------------------------------------------------")
  print ("iteration ", i)
  print ("states in frontier prior to state exploration", len (frontier))
  print (state)

  # 3.2 check if it is final state:
  if is_final_state (state):
    break #we end while. state will remain this last state computed, and sequence of actions will have all states.
  # 3.3 if it is not final, expand.
  else:
    # 3.3.1, 3.3.2__ Our method expand adds all available successors to frontier
    frontier = expand (state, frontier)
  i+=1
#loop keeps running until no more nodes available or final state obtained

# 4. if state is final, the while has broken and we get this solution and print the sequence
if is_final_state (state):
  print (state)
  print ("Steps to final solution: ", len(sequence_of_actions))
  print (sequence_of_actions)
# 5. else
else:
  print ("no solution found")


---------------------------------------------------------------
iteration  1
states in frontier prior to state exploration 0
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
Can't move only farmer
Can't move  Wolf
Can't move  Cabbage
---------------------------------------------------------------
iteration  2
states in frontier prior to state exploration 0
{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
can't move Wolf without farmer
can't move Cabbage without farmer
---------------------------------------------------------------
iteration  3
states in frontier prior to state exploration 1
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
can't move Goat without farmer
---------------------------------------------------------------
iteration  4
states in frontier prior to state exploration 3
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
Can't move only farmer
Can't move  Wolf
Can't move  Cabbage
----------------------------------------------------

In iteration 51 we have the first appearance of a state that leads to final solution: {'Farmer': 'L', 'Wolf': 'R', 'Goat': 'L', 'Cabbage': 'R'}. Farmer is on the left with the goat, so you only need to move the goat and farmer. However, the tree has expanded so much, there are 58 states in the queue, so when we add the two that are a consequenec of expading 51, which are moving only the farmer {'Farmer': 'R', 'Wolf': 'R', 'Goat': 'L', 'Cabbage': 'R'} and moving both  {'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'R'}, we will have 60 states in the queue. So in 60 more movements we will arrive the solution. In the mean time, the tree keeps expanding intermediate states that do not lead to anything, and some loops that have appeared for not controlling the explored states.

**Is this efficient?**
**what options do we have to make it more efficient???**